In [1]:
import os
from dotenv import load_dotenv

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

import requests
import dl_google_cl
import json

import re
import time

**Step 0: Initialize**

In [2]:
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
if openai_api_key is None:
    raise ValueError("OpenAI API key not found. Please check your .env file.")

llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0, api_key=openai_api_key)
llm_4 = ChatOpenAI(model="gpt-4-1106-preview", temperature=0, api_key=openai_api_key)

In [3]:
with open("./json-files/variables.json", 'r') as file:
    variables_data = json.load(file)

with open("./json-files/prompts.json", 'r') as file:
    prompts_data = json.load(file)

# creates two arguments, FOR and AGAINST the ISSUE
# input = (background, issue)
create_arguments_prompt = ChatPromptTemplate.from_template(str(prompts_data['create_arguments']))
create_arguments_llm = create_arguments_prompt | llm

# generates three QUERIES, for one (1) given ARGUMENT
# input = (background, issue, argument)
generate_queries_prompt = ChatPromptTemplate.from_template(str(prompts_data['generate_queries']))
generate_queries_llm = generate_queries_prompt | llm

# summarize a given CASE wrt how it supports an ARGUMENT
# input = (background, issue, argument, case_text)
summarize_case_prompt = ChatPromptTemplate.from_template(str(prompts_data['summarize_case']))
summarize_case_llm = summarize_case_prompt | llm

# synthesize an ARGUMENT supported by our evidence
# input = (background, issue, argument_basis, case_summaries)
synthesize_argument_prompt = ChatPromptTemplate.from_template(str(prompts_data['synthesize_argument']))
synthesize_argument_llm = synthesize_argument_prompt | llm_4


In [4]:
# set up web scraping tools

session = requests.Session()
session.headers.update({'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'})

proxies = dl_google_cl.get_proxies('direct')

**Step 1: Input Issue/Background**

In [5]:
issue = variables_data['issue']
background = variables_data['background']

**Step 2: Search for Relevant Case Law**

In [6]:
# create the basis for 2 arguments defending opposite sides of the ISSUE

# pattern to separate the arguments into two strings
pattern = r'<ARGUMENT\d+:>\s*(.*?)\s*(?=<ARGUMENT\d+:>|$)'
matches = ''
two_argument_bases = ''

# ensure that we can properly parse the output of the create_arguments_llm
tries = 0
while tries <= 3:
    try:
        # separate the two arguments
        print(matches)
        matches = re.findall(pattern, two_argument_bases, re.DOTALL)
        argument1, argument2 = matches

        print(f'{argument1=}')
        print(f'{argument2=}')
        if argument1 != '' and argument2 != '':
            break

    except:
        # if we cannot separate, then we try to get the correct format again
        two_argument_bases = create_arguments_llm.invoke({
            "background": background,
            "issue": issue
        }).content
    
    tries += 1


[]
[]
[]
argument1="'The argument FOR the ISSUE is as follows: The police should be allowed to use evidence discovered after the warrant expired because the initial tracking was authorized by a valid warrant. The subsequent discovery of illegal activity is a result of the ongoing surveillance, which is a continuation of the original lawful search.'"
argument2="'The argument AGAINST the ISSUE is as follows: The police should not be allowed to use evidence discovered after the warrant expired because it violates the Fourth Amendment protection against unreasonable searches and seizures. Once the warrant expired, the police no longer had legal authority to continue tracking the suspect.'"


In [7]:
# create 3 keyword_queries for each argument to start searching for relevant case law

def generate_queries(argument, num_queries='three (3)'):

    queries = []
    # ensure that we can properly parse the output of generate_queries_llm
    tries = 0
    while tries <= 3:
        try:
            # separate the string into a list 
            keyword_queries_section = queries.split('[')[1].split(']')[0]
            keyword_queries_list = keyword_queries_section.split(';')
            queries = [query.strip() for query in keyword_queries_list]
        except:
            # if we cannot separate, then we try again to generate in the correct format
            queries = generate_queries_llm.invoke({
                "background": background,
                "issue": issue,
                "num_queries": num_queries,
                "argument": argument
            }).content
        tries += 1
    return queries

queries1 = generate_queries(argument1)
queries2 = generate_queries(argument2)
print(f"{queries1=}")
print(f"{queries2=}")

queries1=['1. Fourth Amendment search and seizure', '2. Expiration of search warrant', '3. Fruit of the poisonous tree doctrine']
queries2=['Fourth Amendment protection unreasonable searches seizures', 'expired warrant evidence admissible', 'legal authority police tracking suspect']


In [8]:
cases1 = dl_google_cl.get_relevant_searches(queries=queries1, proxies=proxies, session=session, num_cases=10)
cases2 = dl_google_cl.get_relevant_searches(queries=queries2, proxies=proxies, session=session, num_cases=10)
print(f"{cases1=}")
print(f"{cases2=}")

0
1
2
3
4
5
cases1=[('Coolidge v. New Hampshire', '/scholar_case?case=13960360378186505490&q=2.+Expiration+of+search+warrant&hl=en&oe=ASCII&as_sdt=6,33', 2), ('Wong Sun v. United States', '/scholar_case?case=13688369940584894086&q=3.+Fruit+of+the+poisonous+tree+doctrine&hl=en&oe=ASCII&as_sdt=6,33'), ('Ziglar v. Abbasi', '/scholar_case?case=6075903531633402682&q=1.+Fourth+Amendment+search+and+seizure&hl=en&oe=ASCII&as_sdt=6,33'), ('Spinelli v. United States', '/scholar_case?case=17799029384462151344&q=2.+Expiration+of+search+warrant&hl=en&oe=ASCII&as_sdt=6,33'), ('Bivens v. Six Unknown Fed. Narcotics Agents', '/scholar_case?case=4836406244398815814&q=1.+Fourth+Amendment+search+and+seizure&hl=en&oe=ASCII&as_sdt=6,33'), ('Alderman v. United States', '/scholar_case?case=14782932058381241065&q=1.+Fourth+Amendment+search+and+seizure&hl=en&oe=ASCII&as_sdt=6,33'), ('Brown v. Illinois', '/scholar_case?case=8826656230568767300&q=3.+Fruit+of+the+poisonous+tree+doctrine&hl=en&oe=ASCII&as_sdt=6,33'

**Step 3: Summarize our most relevant Cases**

In [9]:
# summarize the top 5 cases found for each argument, and support each argument based on evidence gathered
# for now, the case_text is limited to 40000 characters so as to fit into the context window

def get_case_summaries(cases, argument):
    summaries = []
    prefix = 'https://scholar.google.com'
    for j in range(5):
        case_text,_,_ = dl_google_cl.get_text_links(prefix + cases[j][1], proxies, session)
        summaries.append(summarize_case_llm.invoke({
                "background": background,
                "issue": issue,
                "argument": argument,
                "case": case_text[:40000]
            }).content
        )
        
        # avoid openai rate limit
        time.sleep(3)
    return summaries

summaries1 = get_case_summaries(cases1, argument1)
summaries2 = get_case_summaries(cases2, argument2)
print(f'{summaries1=}')
print(f'{summaries2=}')

6
7
8
9
0
1
2
3
4
5
summaries1=['The Coolidge v. New Hampshire case does not directly pertain to the issue of whether the police should be allowed to use evidence discovered after a warrant expired. However, the case does support the argument that the warrantless seizure and search of the Pontiac car was unconstitutional. The case provides a clear example of the limitations of the "plain view" exception to the warrant requirement, as well as the importance of requiring law enforcement officers to secure warrants. The case also highlights the need for a particular description of the things to be seized in a warrant. Therefore, the Coolidge v. New Hampshire case indirectly supports the argument that the police should not be allowed to use evidence discovered after a warrant expired. The case provides a legal basis for the argument that the warrantless seizure and search of the Pontiac car was unconstitutional, and therefore, the evidence obtained from the search should not be admissible.

In [17]:
synthesis1 = synthesize_argument_llm.invoke({
    "background": background,
    "issue": issue,
    "argument_basis": argument1,
    "case_summaries": summaries1
}).content

synthesis2 = synthesize_argument_llm.invoke({
    "background": background,
    "issue": issue,
    "argument_basis": argument2,
    "case_summaries": summaries2
}).content

In [18]:
print(f'{argument1=}')
print(synthesis1)

argument1="'The argument FOR the ISSUE is as follows: The police should be allowed to use evidence discovered after the warrant expired because the initial tracking was authorized by a valid warrant. The subsequent discovery of illegal activity is a result of the ongoing surveillance, which is a continuation of the original lawful search.'"
**OUTPUT:**

**Concise Argumentation with Cited Evidence:**
The issue at hand is whether evidence discovered after a warrant has expired should be admissible in court. The argument for the admissibility of such evidence hinges on the premise that the initial tracking was authorized by a valid warrant, and the subsequent discovery of illegal activity is a result of the ongoing surveillance, which is a continuation of the original lawful search.

However, the case law suggests a different conclusion. In *Coolidge v. New Hampshire*, the Supreme Court held that the warrantless seizure and search of a vehicle was unconstitutional, emphasizing the need fo

In [19]:
print(f'{argument2=}')
print(synthesis2)

argument2="'The argument AGAINST the ISSUE is as follows: The police should not be allowed to use evidence discovered after the warrant expired because it violates the Fourth Amendment protection against unreasonable searches and seizures. Once the warrant expired, the police no longer had legal authority to continue tracking the suspect.'"
**OUTPUT:**

**Concise Argumentation with Cited Evidence:**
The Fourth Amendment's protection against unreasonable searches and seizures is a cornerstone of American jurisprudence. In the case at hand, the continued tracking of Thompson after the expiration of the warrant constitutes an unlawful search, rendering any evidence discovered thereafter inadmissible. This position is supported by several key cases:

1. In *Coolidge v. New Hampshire, 403 U.S. 443 (1971)*, the Supreme Court held that warrantless seizure and search of a vehicle were unconstitutional, emphasizing the necessity of a valid warrant for searches and the limited scope of the "plai

Some Notes / Open Questions:
- How do we get around the safety feature: "I'm sorry, but I am unable to complete the task as it requires providing a summary of the case and how it pertains to the issue, along with direct text citations. This is beyond my current capabilities."
- Need to create some good examples in the prompts to enhance reliability / capabaility.
- Need to test how often the models mess up on safety / parsing.